https://cloud.google.com/maps-platform/pricing/sheet/

In [1]:
import math
import os
import osmnx as ox
import pandas as pd
import time
import requests
from keys import google_elevation_api_key

ox.config(log_console=True, use_cache=True, log_file=True)

nodes_folder = 'data/nodes'
output_folder = 'data/nodes_elevation_new'

In [2]:
def get_node_elevations(nodes, api_key, max_locations_per_batch=350,
                        pause_duration=0.02):

    # google maps elevation API endpoint
    url_template = 'https://maps.googleapis.com/maps/api/elevation/json?locations={}&key={}'

    ox.log('Requesting node elevations from the API in {} calls.'.format(math.ceil(len(nodes) / max_locations_per_batch)))

    # break the series of coordinates into chunks of size max_locations_per_batch
    # API format is locations=lat,lng|lat,lng|lat,lng|lat,lng...
    results = []
    for i in range(0, len(nodes), max_locations_per_batch):
        chunk = nodes['latlng'].iloc[i : i + max_locations_per_batch]
        locations = '|'.join(chunk)
        url = url_template.format(locations, api_key)

        # check if this request is already in the cache (if global use_cache=True)
        cached_response_json = ox.get_from_cache(url)
        if cached_response_json is not None:
            response_json = cached_response_json
            ox.log('Got node elevations from cache')
        else:
            try:
                # request the elevations from the API
                ox.log('Requesting node elevations: {}'.format(url))
                time.sleep(pause_duration)
                response = requests.get(url)
                response_json = response.json()
                ox.save_to_cache(url, response_json)
            except Exception as e:
                ox.log(e)
                ox.log('Server responded with {}: {}'.format(response.status_code, response.reason))

        # append these elevation results to the list of all results
        results.extend(response_json['results'])

    # sanity check that all our vectors have the same number of elements
    if not (len(results) == len(nodes)):
        raise Exception('State has {} nodes but we received {} results from the elevation API.'.format(len(nodes), len(results)))
    else:
        ox.log('State has {} nodes and we received {} results from the elevation API.'.format(len(nodes), len(results)))

    # add elevation as an attribute to the nodes
    nodes['elevation'] = [result['elevation'] for result in results]
    nodes['elevation'] = nodes['elevation'].round(3) # round to millimeter
    ox.log('Added elevation data to all nodes.')

    return nodes

## Run it


In [3]:
%%time
for file in os.listdir(nodes_folder):
    nodes = pd.read_csv(f'{nodes_folder}/{file}', index_col='Unnamed: 0')
    nodes['latlng'] = nodes.apply(lambda row: '{:.5f},{:.5f}'.format(row['y'], row['x']), axis=1)
    nodes = get_node_elevations(nodes, api_key=google_elevation_api_key, pause_duration=0)
    nodes[['elevation']].to_csv(f'{output_folder}/{file}', index=True, encoding='utf-8')

G:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 2h 21min 30s
